In [1]:
# Install required libraries
# %pip install crewai crewai_tools langchain langchain_community langchain_ollama streamlit duckduckgo-search

# ✅ **Library Installation** 🚀  

You have to successfully installed all the required dependencies. Below is a breakdown of the installed libraries and their purpose:  

## 📦 **Installed Libraries & Their Uses**  

| 🔹 Library | 🔍 Purpose |
|------------|-----------|
| **`crewai`** | AI-powered workflow automation. |
| **`crewai_tools`** | Additional utilities for `crewai` workflows. |
| **`langchain`** | Framework for integrating LLMs into applications. |
| **`langchain_community`** | Community-driven extensions for `langchain`. |
| **`langchain_ollama`** | Enables seamless interaction with locally running models. |
| **`streamlit`** | Web-based framework for building AI applications. |
| **`duckduckgo-search`** | Enables search integration using DuckDuckGo. |

In [3]:
from crewai import Agent, Task, Crew, Process, LLM
from crewai.tools import tool
from langchain_community.tools import DuckDuckGoSearchResults

# **=== 1️⃣ Define the Web Search Tool ===**

In [5]:
@tool
def search_web_tool(query: str):
    """Searches the web for relevant travel information."""
    search_tool = DuckDuckGoSearchResults(num_results=10, verbose=True)
    return search_tool.run(query)

# **=== 2️⃣ Define LLM Connection ===**

In [7]:
llm = LLM(model="ollama/llama3.2", base_url="http://localhost:11434")

# **=== 3️⃣ Define Agents ===**

In [9]:
# Agent 1: Location Expert
location_expert = Agent(
    role="Travel Location Expert",
    goal="Gather information about the city, accommodations, weather, travel advisories.",
    backstory="A seasoned travel researcher providing insights on where to stay, what to expect, and travel tips.",
    tools=[search_web_tool],
    verbose=True,
    max_iter=5,
    llm=llm,
    allow_delegation=False,
)

# Agent 2: Guide Expert
guide_expert = Agent(
    role="Local Guide Expert",
    goal="Provides recommendations for sightseeing, food, and experiences.",
    backstory="A passionate local guide with knowledge of the best spots in town.",
    tools=[search_web_tool],
    verbose=True,
    max_iter=5,
    llm=llm,
    allow_delegation=False,
)

# Agent 3: Travel Planner Expert
planner_expert = Agent(
    role="Travel Planner Expert",
    goal="Creates a complete travel itinerary based on location and guide inputs.",
    backstory="A meticulous travel planner who ensures a seamless experience.",
    tools=[search_web_tool],
    verbose=True,
    max_iter=5,
    llm=llm,
    allow_delegation=False,
)

# **=== 4️⃣ Define Tasks ===**

In [11]:
def create_location_task(agent, from_city, destination_city, date_from, date_to):
    return Task(
        description=f"""
        Gather **detailed travel insights** for {destination_city}.
        - Departure: {from_city}
        - Destination: {destination_city}
        - Dates: {date_from} to {date_to}

        **Must Include:**
        - Best hotels, hostels, and Airbnb recommendations with pricing
        - Daily cost breakdown: accommodation, food, transport, entertainment
        - Visa and travel restrictions
        - Weather forecast and seasonal travel tips
        - Hidden costs or common tourist scams

        **Expand each section with comprehensive details (at least 1-2 pages).**
        """,
        expected_output="""
        A **detailed 1-2 page markdown report** covering:
        - **Accommodations** (budget to luxury)
        - **Expenses** (itemized cost breakdown)
        - **Travel Tips** (visa, safety, weather insights)
        """,
        agent=agent,
        output_file="city_report.md",
    )

def create_guide_task(agent, destination_city, interests, date_from, date_to):
    return Task(
        description=f"""
        Generate a **detailed travel guide** for {destination_city} based on these interests: {interests}.
        - Dates: {date_from} to {date_to}

        **Include:**
        - **Top attractions and must-visit places**
        - **Local cuisine recommendations (street food to fine dining)**
        - **Entertainment options: nightlife, shopping, cultural festivals**
        - **Hidden gems: unique places tourists often miss**
        - **Safety tips, best transportation, and city navigation hacks**

        **Ensure each section is well-detailed for a total of 1-2 pages.**
        """,
        expected_output="""
        A **1-2 page markdown travel guide** including:
        - **Attractions & Experiences**
        - **Dining & Food**
        - **Hidden Gems & Local Secrets**
        - **Best Travel Tips for Visitors**
        """,
        agent=agent,
        output_file="guide_report.md",
    )

def create_planner_task(context, agent, destination_city, interests, date_from, date_to):
    return Task(
        description=f"""
        Create a **detailed travel itinerary** for {destination_city}.
        - Interests: {interests}
        - Travel Dates: {date_from} to {date_to}

        **Must Cover:**
        - **Introduction to {destination_city}** (cultural highlights, history, why visit?)
        - **Estimated budget breakdown** (daily expenses, total trip cost)
        - **Full daily itinerary** (morning, afternoon, evening plans)
        - **Local transport options & travel routes**
        - **Packing tips based on weather and cultural norms**
        
        **Ensure detailed formatting and structured breakdown (1-2 pages).**
        """,
        expected_output="""
        A **structured 1-2 page markdown itinerary** including:
        - **City Overview**
        - **Cost Breakdown**
        - **Detailed Daily Plan**
        - **Transport & Navigation Tips**
        """,
        context=context,
        agent=agent,
        output_file="travel_plan.md",
    )

# **=== 5️⃣ User Input for Dynamic Travel Planning ===**

In [13]:
from_city = input("Enter your departure city: ")
destination_city = input("Enter your destination city: ")
date_from = input("Enter departure date (e.g., 1st April 2025): ")
date_to = input("Enter return date (e.g., 10th April 2025): ")
interests = input("Enter travel interests (e.g., food, sightseeing, adventure): ")

Enter your departure city:  Delhi
Enter your destination city:  Tokyo
Enter departure date (e.g., 1st April 2025):  12th December 2025
Enter return date (e.g., 10th April 2025):  20th December 2025
Enter travel interests (e.g., food, sightseeing, adventure):  food, sightseeing, adventure


# **=== 6️⃣ Create Tasks Dynamically ===**

In [15]:
location_task = create_location_task(location_expert, from_city, destination_city, date_from, date_to)
guide_task = create_guide_task(guide_expert, destination_city, interests, date_from, date_to)
planner_task = create_planner_task([location_task, guide_task], planner_expert, destination_city, interests, date_from, date_to)

# **=== 7️⃣ Setup CrewAI Execution ===**

In [17]:
crew = Crew(
    agents=[location_expert, guide_expert, planner_expert],
    tasks=[location_task, guide_task, planner_task],
    process=Process.sequential,
    full_output=True,
    share_crew=False,
    verbose=True
)

# **=== 8️⃣ Run CrewAI to Generate Reports ===**

In [19]:
result = crew.kickoff()
print("\n✅ Travel reports generated successfully!\n")

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 7a3fd86a-ed4c-4447-8def-0d834d30a307                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 75364635-0040-4a4b-9d1e-395f2a94b9c2
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 75364635-0040-4a4b-9d1e-395f2a94b9c2
       Status: Executing Task...
    └── 🤖 Agent: Travel Location Expert
            Status: In Progress

# Agent: Travel Location Expert
## Task: 
        Gather **detailed travel insights** for Tokyo.
        - Departure: Delhi
        - Destination: Tokyo
        - Dates: 12th December 2025 to 20th December 2025

        **Must Include:**
        - Best hotels, hostels, and Airbnb recommendations with pricing
        - Daily cost breakdown: accommodation, food, transport, entertainment
        - Visa and travel restrictions
        - Weather forecast and seasonal travel tips
        - Hidden costs or common tourist scams

        **Expand each section with comprehensive details (at least 1-2 pages).**
        


🚀 Crew: crew
└── 📋 Task: 75364635-0040-4a4b-9d1e-395f2a94b9c2
       Status: Executing Task...
    └── 🤖 Agent: Travel Location Expert
            Status: In Progress
        └── 🧠 Thinking...

...

🚀 Crew: crew
├── 📋 Task: 75364635-0040-4a4b-9d1e-395f2a94b9c2
│      Assigned to: Travel Location Expert
│      Status: ✅ Completed
│   └── 🤖 Agent: Travel Location Expert
│           Status: ✅ Completed
│       └── 🔧 Used search_web_tool (4)
├── 📋 Task: bb2d3ed8-fd35-462e-b863-21e0a867dd90
│      Assigned to: Local Guide Expert
│      Status: ✅ Completed
│   └── 🤖 Agent: Local Guide Expert
│           Status: ✅ Completed
│       └── 🔧 Used search_web_tool (5)
└── 📋 Task: 390646dd-94f2-4b8d-99d3-34e365ccfbae
       Status: Executing Task...
    └── 🤖 Agent: Travel Planner Expert
            Status: ✅ Completed
        └── 🔧 Used search_web_tool (6)

🚀 Crew: crew
├── 📋 Task: 75364635-0040-4a4b-9d1e-395f2a94b9c2
│      Assigned to: Travel Location Expert
│      Status: ✅ Completed
│   └── 🤖 Agent: Travel Location Expert
│           Status: ✅ Completed
│       └── 🔧 Used search_web_tool (4)
├── 📋 Task: bb2d3ed8-fd35-462e-b863-21e0a867dd90
│      Assigned to: Local Guide Expert
│      Status: ✅ Completed
│   └── 🤖 Agent: Local Guide Expert
│           Status: ✅ Completed
│       └── 🔧 Used search_web_tool (5)
└── 📋 Task: 390646dd-94f2-4b8d-99d3-34e365ccfbae
       Assigned to: Travel Planner Expert
       Status: ✅ Completed
    └── 🤖 Agent: Travel Planner Expert
            Status: ✅ Completed
        └── 🔧 Used search_web_tool (6)

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 390646dd-94f2-4b8d-99d3-34e365ccfbae                                                                     │
│  Agent: Travel Planner Expert                                                                                   │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 7a3fd86a-ed4c-4447-8def-0d834d30a307                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


✅ Travel reports generated successfully!

